<body style="background-color:powderblue;">
<h1 style="text-align:center; font-family:serif; font-size:300%">AES Key Generation</h1>

<p style="text-align:center; font-family:courier; font-size:160%">Prashanth.S 19MID0020</p>

 <h1 style="font-family:serif; font-size:200%">Importing the Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import ast
import itertools
import collections
import struct
import sys

 <h1 style="font-family:serif; font-size:200%">Importing the key S-Box 

In [2]:
col_names  = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']
row_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

enc_key_sbox = pd.read_excel('AES tables.xlsx', index_col=0)

enc_key_sbox.columns = col_names ## replacing the column names 
enc_key_sbox.index = row_names    ## replacing the row names

enc_key_sbox.head()

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
0,63,7c,77,7b,f2,6b,6f,c5,30,1,67,2b,fe,d7,ab,76
1,ca,82,c9,7d,fa,59,47,f0,ad,d4,a2,af,9c,a4,72,c0
2,b7,fd,93,26,36,3f,f7,cc,34,a5,e5,f1,71,d8,31,15
3,4,c7,23,c3,18,96,5,9a,7,12,80,e2,eb,27,b2,75
4,9,83,2c,1a,1b,6e,5a,a0,52,3b,d6,b3,29,e3,2f,84


 <h1 style="font-family:serif; font-size:200%">Getting inputs

In [3]:
def text_hexadecimal(text): ## all the blocks --> 16bytes
    hex_text   = []
    for i in text: ## character -> ascii (decimal) -> hexa-decimal
        hex_text.append(hex(ord(i))[2:])
    return hex_text ## 16-byte representation of the text

In [4]:
key_str = 'Thats my Kung Fu' 
hex_key = text_hexadecimal(key_str)
print("Hexadecimal Representation : ",hex_key)

Hexadecimal Representation :  ['54', '68', '61', '74', '73', '20', '6d', '79', '20', '4b', '75', '6e', '67', '20', '46', '75']


In [5]:
## splitting into 4*4 matrix
hex_key = [hex_key[i:i+4] for i in range(0, len(hex_key), 4)]
hex_key

[['54', '68', '61', '74'],
 ['73', '20', '6d', '79'],
 ['20', '4b', '75', '6e'],
 ['67', '20', '46', '75']]

In [6]:
complete_keys = [] ## to accomodate keys
complete_keys.append(hex_key)

 <h1 style="font-family:serif; font-size:200%">Key Generation

In [7]:
def key_sbox(element):
    row_index = element[0]
    col_index  = element[1]
    ans = enc_key_sbox.loc[[row_index],[col_index]][col_index][0]
    return str(ans)

In [8]:
def binaryToDecimal(binary):
    binary1 = binary
    decimal, i, n = 0, 0, 0
    
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)
        binary = binary//10
        i += 1
        
    hexadecimal =  hex(decimal)[-1]
    return hexadecimal

In [9]:
def bcd_hexadecimal(bcd_list):
    w4 = []
    
    for i in range(0,len(bcd_list),2):
        temp_str = ' '
        temp_str = binaryToDecimal(int(bcd_list[i]))
        temp_str = temp_str + binaryToDecimal(int(bcd_list[i+1]))
        w4.append(temp_str)
        
    return w4

In [10]:
def HexaDecimaltoBCD(str):
    list1 = []
    for i in range(len(str)):
        decimal = int(str[i], 16)                                ## hexadecimal -> decimal
        binary_num = bin(decimal).replace("0b", "")  # decimal -> binary
        list1.append(binary_num)
            
    ## binary in-terms of 4 bits
    for i in range(len(list1)):
        element = list1[i]
        if len(element)<4:
            diff = 4 - len(element)
            for j in range(diff):
                element = "0" + element
            list1[i] = element
            
    return list1

In [11]:
def Hexaword_BCD(list1):
    bcd = [ HexaDecimaltoBCD(i) for i in list1 ]
    bcd = list(np.concatenate(bcd).flat)    ## 2d list to 1d list
    return bcd

In [12]:
def XOR(list1, list2):
    
    def compare(element1, element2):
        ans = []
        for i,j in zip(element1,element2):
            if (i!=j):ans.append(1)
            else:ans.append(0)
        return ans

    main_ans = []
    for i in range(len(list1)):
        main_ans.append(compare(list1[i], list2[i]))
        
    main_ans = [ "".join(list(map(str, i))) for i in main_ans ]
    return main_ans

In [13]:
def rotword(word):
    rot=word[1:]
    rot.append(word[0])
    return rot

In [14]:
## round-constant table 
## key-round and value-hexadecimal
round_constant = {1:'1', 2:'2', 3:'4', 4:'8', 5:'10', 6:'20', 7:'40', 8:'80', 9:'1B', 10:'36'}

In [15]:
def col1_generation(iteration_var, hex_key):

    ## taking the last column words
    last_col = hex_key[-1]

    ## left shifting the last column words
    #left_shift = last_col[1:] + last_col[:1] 
    left_shift = rotword(last_col)

    ## sub-word generation from S-Box
    subword = []
    for i in left_shift:
        val = key_sbox(i)
        if len(val)!=2:
            val = '0' + val
            subword.append(val)
        else:
            subword.append(val)

    ## subword --> hexadecimal(subword)
    y1 = Hexaword_BCD(subword)

    ## subword (XOR) Round Constant
    element = round_constant[iteration_var]
    initial = HexaDecimaltoBCD(element)
    initial_length = len(initial)

    if (initial_length == 1):
        between = initial
        temp_1 = [ [0 for j in range(4) for i in range(1)] ]
        last = [ [0 for j in range(4)] for i in range(6) ]
        temp_1.extend(between)
        temp_1.extend(last)
        final = temp_1
        if (element==1): ## if there is a single element(0,1,2, ... ,9) from the s-box --> length=1
            before1 = [ [0 for j in range(4) for i in range(1)] ]
            before1.extend(final)
            final = before1

    elif (initial_length == 2):  ## if there is a two element(11, 1a, b1, ...) from the s-box --> length=2
        last = [ [0 for j in range(4)] for i in range(6) ]
        first = initial
        first.extend(last)
        final = first

    final = [ "".join(list(map(str, i))) for i in final ]
    round_list = final

    round_ans = XOR(y1, round_list) ## --> g(col4_before)
    print("y1 : ",y1)
    # taking col1_before
    col1_before = [ HexaDecimaltoBCD(i) for i in hex_key[0] ]
    col1_before = list(np.concatenate(col1_before).flat)

    ## col1 --> col1_before (exor) g(col4_before)
    col1 = XOR(col1_before, round_ans)
    col1 = bcd_hexadecimal(col1)

    return col1

In [16]:
for i in range(1,11): ## 10 times running
    temp = []
    
    ## 1st col word = col_before[0] (exor) g(col_before[-1]) 
    col1_hex = col1_generation(i, hex_key)
    col1_bin  = Hexaword_BCD(col1_hex)
    
    for j in range(2,5): ## each loop, 3 times running ( remaining 3 words)
        if (j==2):  ## 2nd col words
            col1_before_bin = Hexaword_BCD(hex_key[j-1])
            col2_bin = XOR(col1_before_bin, col1_bin)
            col2_hex = bcd_hexadecimal(col2_bin)
        
        else:  ## 3rd and 4th column words
            col2_before_bin = Hexaword_BCD(hex_key[j-1])            
            col3_bin = XOR(col2_before_bin, col2_bin)
            col3_hex = bcd_hexadecimal(col3_bin)
            col2_bin = Hexaword_BCD(col3_hex)
            temp.append(col3_hex)
    
    ## once again generating the hex_key
    hex_key = []
    
    hex_key.append(col1_hex)
    hex_key.append(col2_hex)
    hex_key.extend(temp)    
    
    complete_keys.append(hex_key)

y1 :  ['1011', '0111', '0101', '1010', '1001', '1101', '1000', '0101']
y1 :  ['1011', '0110', '0011', '1010', '1101', '1100', '1111', '0110']
y1 :  ['1000', '0000', '0110', '1000', '0010', '1101', '1110', '0000']
y1 :  ['0111', '1011', '0111', '0010', '0000', '1111', '0010', '1110']
y1 :  ['0000', '0000', '0011', '1011', '0011', '1001', '1111', '1010']
y1 :  ['0010', '1100', '0001', '0100', '1111', '1001', '1011', '0100']
y1 :  ['0011', '0001', '1010', '1011', '0010', '1111', '1001', '0001']
y1 :  ['1100', '0010', '1100', '0111', '0000', '0010', '0011', '0111']
y1 :  ['0010', '1010', '1011', '0011', '0101', '0000', '1011', '0001']
y1 :  ['1010', '0001', '0001', '1111', '0110', '0001', '0110', '1000']


In [38]:
def binary_division_modulo_2(val1, val2):

    def showpoly(a):
        str1 = ""
        nobits = len(a)
        for x in range (0,nobits-2):
            if (a[x] == '1'):
                if (len(str1)==0):str1 +="x**"+str(nobits-x-1)	
                else: str1 +="+x**"+str(nobits-x-1)
        if (a[nobits-2] == '1'):
            if (len(str1)==0):str1 +="x"
            else:str1 +="+x"

        if (a[nobits-1] == '1'):str1 +="+1"
        print(str1)

    def toList(x):
        l = []
        for i in range (0,len(x)):l.append(int(x[i]))
        return (l)

    def toString(x):
        str1 =""
        for i in range (0,len(x)):str1+=str(x[i])
        return (str1)

    def divide(val1,val2):
        a = toList(val1)
        b = toList(val2)
        working=""
        res=""
        
        while len(b) <= len(a) and a:
            if (a[0] == 1):
                del a[0]
                for j in range(len(b)-1):
                    a[j] ^= b[j+1]
                if (len(a)>0):
                    working +=toString(a)
                    res+= "1"
                else:
                    del a[0]
                    working +=toString(a)
                    res+="0"
        return res, a
    
    val1, val2 = divide(val1,val2)

    ## checking the length
    val2_length = len(val2)
    if (val2_length<8):
        for i in range(8-val2_length):val2.insert(0,0)
    
    ## convert 4*4 pattern
    str1 = ''
    for i in val2: str1 = str1 + str(i)
    remainder = str1
    return remainder

In [39]:
plain_text_str = 'Two One Nine Two' 
hex_str = text_hexadecimal(plain_text_str)
hex_str = [hex_str[i:i+4] for i in range(0, len(hex_str), 4)]
print("Hexadecimal Representation : ",hex_str)

Hexadecimal Representation :  [['54', '77', '6f', '20'], ['4f', '6e', '65', '20'], ['4e', '69', '6e', '65'], ['20', '54', '77', '6f']]


In [40]:
add_round_key = []
for i in range(4):
    bin_str = Hexaword_BCD(hex_str[i])
    bin_keys = Hexaword_BCD(complete_keys[0][i])
    bin_xor   = XOR(bin_str, bin_keys)
    bcd_round_key = bcd_hexadecimal(bin_xor)
    add_round_key.append(bcd_round_key)
add_round_key

[['00', '1f', '0e', '54'],
 ['3c', '4e', '08', '59'],
 ['6e', '22', '1b', '0b'],
 ['47', '74', '31', '1a']]

In [41]:
subword = []
for i in range(4):
    temp_word = []
    for j in add_round_key:
        val = key_sbox(j[i])
        if len(val)!=2:
            val = '0' + val
            temp_word.append(val)
        else:
            temp_word.append(val)

    ## subword --> hexadecimal(subword)
    temp_word = Hexaword_BCD(temp_word)
    temp_word_hexa = bcd_hexadecimal(temp_word)
    subword.append(temp_word_hexa)
    temp_word_hexa = []

## swapping rows and columns
#subword = [list(x) for x in zip(*subword)]
subword

[['63', 'eb', '9f', 'a0'],
 ['c0', '2f', '93', '92'],
 ['ab', '30', 'af', 'c7'],
 ['20', 'cb', '2b', 'a2']]

In [42]:
shift_rows = []
subword = collections.deque(subword)
for i in range(4):
    temp = collections.deque(subword[i])
    temp.rotate(-i)
    shift_rows.append(list(temp))
shift_rows

[['63', 'eb', '9f', 'a0'],
 ['2f', '93', '92', 'c0'],
 ['af', 'c7', 'ab', '30'],
 ['a2', '20', 'cb', '2b']]

In [43]:
shift_rows = [list(x) for x in zip(*shift_rows)]
shift_rows

[['63', '2f', 'af', 'a2'],
 ['eb', '93', 'c7', '20'],
 ['9f', '92', 'ab', 'cb'],
 ['a0', 'c0', '30', '2b']]

In [44]:
multiple = [['02', '03', '01', '01'],
                 ['01', '02', '03', '01'],
                 ['01', '01', '02', '03'],
                 ['03', '01', '01', '02']]

In [45]:
def bin_product(list1, list2):
    product = []
    for i,j in zip(list1, list2):
        ans = int(i,2) * int(j,2)
        print(ans)
        product.append(ans)
    return product

In [46]:
multiple

[['02', '03', '01', '01'],
 ['01', '02', '03', '01'],
 ['01', '01', '02', '03'],
 ['03', '01', '01', '02']]

In [47]:
shift_rows

[['63', '2f', 'af', 'a2'],
 ['eb', '93', 'c7', '20'],
 ['9f', '92', 'ab', 'cb'],
 ['a0', 'c0', '30', '2b']]

In [53]:
final_prod_ans = []    
prod_ans = []

for i,j in zip(multiple, shift_rows):

    for k in range(4):
        
        ## hexadecimal to BCD
        num1 = Hexaword_BCD(i[k])
        num2 = Hexaword_BCD(j[k])
        print("num1 : {} and num2: {} ".format(i[k], j[k]))
        
        ## Product operation
        ## concatenating the 1st and 2nd index of the list
        num1_join = int( (str(num1[0]) + str(num1[1])) ,2)  ## concatenated bcd to decimal
        num2_join = int( (str(num2[0]) + str(num2[1])) ,2)  ## concatenated bcd to decimal
        
        ans = bin(num1_join * num2_join)[2:]   ## decimal product and back to bcd
        
        if (len(ans)>8): ## Not in the limit of GF(2^8) --> So Divide by an ir-reducible polynomial
            irreducible_polynomial = '100011011'
            ans = binary_division_modulo_2(irreducible_polynomial, ans)
            prod_ans.append(ans)
        elif len(ans)==8: ## Within the limit of GF(2^8) --> No Need to divide by an ir-reducible polynomial
            prod_ans.append(ans)
    
    #print("prod_ans : ",prod_ans)
    #print("\n")
        
    ## XOR operation
    xor_ans = []
    temp_ans = []
    
    ans34 = XOR(prod_ans[-4], prod_ans[-3])
    ans12 = XOR(prod_ans[-2], prod_ans[-1])
    temp_ans.append(XOR(ans12, ans34))
    
    ## flattening the list  now the list of individual 8 elements
    temp_ans = [item for sublist in temp_ans for item in sublist] 
    
    ## separating 4,4 pattern
    xor_ans.append(''.join(temp_ans)[:4])
    xor_ans.append(''.join(temp_ans)[4:])
    final_prod_ans.append(bcd_hexadecimal(xor_ans))
    
    print(*bcd_hexadecimal(xor_ans))
    print("\n")

num1 : 02 and num2: 63 
num1 : 03 and num2: 2f 
num1 : 01 and num2: af 
num1 : 01 and num2: a2 
46


num1 : 01 and num2: eb 
num1 : 02 and num2: 93 
num1 : 03 and num2: c7 
num1 : 01 and num2: 20 
f9


num1 : 01 and num2: 9f 
num1 : 01 and num2: 92 
num1 : 02 and num2: ab 
num1 : 03 and num2: cb 
cd


num1 : 03 and num2: a0 
num1 : 01 and num2: c0 
num1 : 01 and num2: 30 
num1 : 02 and num2: 2b 
fb




In [28]:
final_prod_ans

[['46'], ['f9'], ['cd'], ['fb']]

In [58]:
for i,j in zip(multiple, shift_rows):
    for k in range(4):
        print("num1 : {} and num2: {} ".format(i[k], j[k]))
    print("\n")

num1 : 02 and num2: 63 
num1 : 03 and num2: 2f 
num1 : 01 and num2: af 
num1 : 01 and num2: a2 


num1 : 01 and num2: eb 
num1 : 02 and num2: 93 
num1 : 03 and num2: c7 
num1 : 01 and num2: 20 


num1 : 01 and num2: 9f 
num1 : 01 and num2: 92 
num1 : 02 and num2: ab 
num1 : 03 and num2: cb 


num1 : 03 and num2: a0 
num1 : 01 and num2: c0 
num1 : 01 and num2: 30 
num1 : 02 and num2: 2b 




In [55]:
shift_rows

[['63', '2f', 'af', 'a2'],
 ['eb', '93', 'c7', '20'],
 ['9f', '92', 'ab', 'cb'],
 ['a0', 'c0', '30', '2b']]

In [56]:
multiple

[['02', '03', '01', '01'],
 ['01', '02', '03', '01'],
 ['01', '01', '02', '03'],
 ['03', '01', '01', '02']]

In [29]:
val1 = '100011011'
val2 = '100100110'

remainder = binary_division_modulo_2(val1, val2)

#print(quotient)
print(remainder)

00111101


 <h1 style="font-family:serif; font-size:200%">Keys for all rounds

In [30]:
for i in range(len(complete_keys)):
    print("Round - {} keys --> ".format(i+1),end=' ')
    print(*list(itertools.chain(*complete_keys[i])))

Round - 1 keys -->  54 68 61 74 73 20 6d 79 20 4b 75 6e 67 20 46 75
Round - 2 keys -->  e2 32 fc f1 91 12 91 88 b1 59 e4 e6 d6 79 a2 93
Round - 3 keys -->  56 08 20 07 c7 1a b1 8f 76 43 55 69 a0 3a f7 fa
Round - 4 keys -->  d2 60 0d e7 15 7a bc 68 63 39 e9 01 c3 03 1e fb
Round - 5 keys -->  a1 12 02 c9 b4 68 be a1 d7 51 57 a0 14 52 49 5b
Round - 6 keys -->  b1 29 3b 33 05 41 85 92 d2 10 d2 32 c6 42 9b 69
Round - 7 keys -->  bd 3d c2 87 b8 7c 47 15 6a 6c 95 27 ac 2e 0e 4e
Round - 8 keys -->  cc 96 ed 16 74 ea aa 03 1e 86 3f 24 b2 a8 31 6a
Round - 9 keys -->  8e 51 ef 21 fa bb 45 22 e4 3d 7a 06 56 95 4b 6c
Round - 10 keys -->  bf e2 bf 90 45 59 fa b2 a1 64 80 b4 f7 f1 cb d8
Round - 11 keys -->  28 fd de f8 6d a4 24 4a cc c0 a4 fe 3b 31 6f 26


In [31]:
complete_keys[0]

[['54', '68', '61', '74'],
 ['73', '20', '6d', '79'],
 ['20', '4b', '75', '6e'],
 ['67', '20', '46', '75']]

In [32]:
def add_round_key(hex_str, hex_key): ## exor with plain text and key
    
    add_round_key = []
    for i in range(4):
        bin_str = Hexaword_BCD(hex_str[i])
        bin_keys = Hexaword_BCD(complete_keys[0][i])
        bin_xor   = XOR(bin_str, bin_keys)
        bcd_round_key = bcd_hexadecimal(bin_xor)
        add_round_key.append(bcd_round_key)

        return add_round_key

In [33]:
def subtitute_box(add_round_key):  ## output from add_round_key
    subword = []
    for i in range(4):
        temp_word = []
        for j in add_round_key:
            val = key_sbox(j[i])
            if len(val)!=2:
                val = '0' + val
                temp_word.append(val)
            else:
                temp_word.append(val)

        ## subword --> hexadecimal(subword)
        temp_word = Hexaword_BCD(temp_word)
        temp_word_hexa = bcd_hexadecimal(temp_word)
        subword.append(temp_word_hexa)
        temp_word_hexa = []

    ## swapping rows and columns
    subword = [list(x) for x in zip(*subword)]
    return subword

In [34]:
def shift_rows(subword):
    shift_rows = []
    subword = collections.deque(subword)
    for i in range(4):
        temp = collections.deque(subword[i])
        temp.rotate(-i)
        shift_rows.append(list(temp))
    return shift_rows

In [35]:
def mix_columns():

IndentationError: expected an indented block (3303023366.py, line 1)

In [ ]:
def main():
    round_num = 0

In [ ]:
print(hex_str)

In [ ]:
print(complete_keys[0])

In [ ]:
XOR(hex_str,complete_keys[0])

In [ ]:
for i,j in zip(hex_str, complete_keys[0]):
    print(i[0],j[0])
    #print(XOR(i[0],j[0]))
    
    #print(Hexaword_BCD(i[0]))
    
    col1_before = [ HexaDecimaltoBCD(element) for element in i[0] ]
    col1_before = list(np.concatenate(col1_before).flat)
    
    print(col1_before)
    break

In [ ]:
#col1_hex = col1_generation(i, hex_key)
col1_bin  = Hexaword_BCD(hex_str)
print(col1_bin)

In [ ]:
col1_before = [ HexaDecimaltoBCD(i) for i in hex_key[0] ]
col1_before = list(np.concatenate(col1_before).flat)

In [ ]:
    col1_before = [ HexaDecimaltoBCD(element) for element in i ]
    col1_before = list(np.concatenate(col1_before).flat)
    print(col1_before)

<h1 style="font-family:serif; font-size:150%">Mam, I have included my code here


https://github.com/PrashanthSingaravelan/winter_semester-2022-assignments/tree/main/CSI3002%20Applied%20Cryptography%20and%20Network%20Security/lab%20assignment/assignment_3

In [ ]:
'''
https://asecuritysite.com/comms/mod_div?a=10011&b=11
'''